<a href="https://colab.research.google.com/github/skywalker0803r/oil_web/blob/master/%E9%96%8B%E7%99%BC%E9%81%8E%E7%A8%8B%E6%96%87%E4%BB%B6/%E7%AC%AC%E5%9B%9B%E7%89%88%E6%A8%A1%E7%B5%84%E9%96%8B%E7%99%BC_SVR_%E5%BA%8F%E5%88%97%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
from sklearn.svm import SVR
from tqdm import tqdm_notebook as tqdm
import numpy as np
from sklearn.metrics import r2_score,mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler
import warnings;warnings.simplefilter('ignore')
from sklearn.pipeline import Pipeline
import joblib
from scipy.special import softmax
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# load data

In [0]:
def load_data():
  train = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/phase1/train_4565.csv',index_col=0)
  test = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/phase1/test_170.csv',index_col=0)
  
  # add P
  train['P'] = 100 - train['N+A']
  test['P'] = 100 - test['N+A']
  
  # define columns
  x_cols = ['T10','T50','T90','N+A','P']
  y_cols = ['C5NP','C5IP','C5N','C6NP','C6IP','C6N','C6A','C7NP','C7IP','C7N','C7A',
            'C8NP','C8IP','C8N','C8A','C9NP','C9IP','C9N','C9A','C10NP','C10IP','C10N','C10A']
  N_col = ['C5N','C6N','C6A','C7N','C7A','C8N','C8A','C9N','C9A','C10N','C10A']
  P_col = ['C5NP','C5IP','C6NP','C6IP','C7NP','C7IP','C8NP','C8IP','C9NP','C9IP','C10NP','C10IP']
  
  # split_data
  X_train = train[x_cols]
  X_test = test[x_cols]
  y_train = train[y_cols]
  y_test = test[y_cols]
  
  return X_train,y_train,X_test,y_test

In [0]:
X_train,y_train,X_test,y_test = load_data()

# define model

In [0]:
class custom_model(object):
  def __init__(self,x_cols,y_cols):
    self.x_cols = x_cols
    self.y_cols = y_cols
    self.N_col = ['C5N','C6N','C6A','C7N','C7A','C8N','C8A','C9N','C9A','C10N','C10A']
    self.P_col = ['C5NP','C5IP','C6NP','C6IP','C7NP','C7IP','C8NP','C8IP','C9NP','C9IP','C10NP','C10IP']
    # initialize models
    self.model_23 = {}
    for y_name in y_cols:
      self.model_23[y_name] = Pipeline([('scaler',StandardScaler()),('reg',SVR(C=0.3))])
  
  def fit(self,X,y):
    for y_name in tqdm(self.y_cols):
      self.model_23[y_name].fit(X,y[y_name])
      y_pred = self.model_23[y_name].predict(X) 
      # 序列預測把預測值併入特徵
      X.loc[:,y_name] = y_pred
    #最後變回來
    X = X[self.x_cols]
  
  def predict(self,X):
    results = pd.DataFrame(index=[*range(len(X))],columns=self.y_cols)
    
    #依序預測23個成份
    for y_name in self.y_cols:
      y_pred = self.model_23[y_name].predict(X)
      results.loc[:,y_name] = y_pred
      #序列預測把預測值併入特徵
      X.loc[:,y_name] = y_pred
    # 最後變回來
    X = X[self.x_cols]
    
    # 記得normalize
    results[self.N_col] = self._normalize(results[self.N_col])*X['N+A'].values.reshape(-1,1)
    results[self.P_col] = self._normalize(results[self.P_col])*X['P'].values.reshape(-1,1)

    return results.values
  
  @staticmethod
  def _normalize(x):
    return x/x.sum(axis=1).values.reshape(-1,1)

# fit model

In [5]:
x_cols = X_train.columns.tolist()
y_cols = y_train.columns.tolist()
model = custom_model(x_cols,y_cols)
model.fit(X_train,y_train)

# model predict

In [6]:
y_pred = model.predict(X_test)
y_pred.shape

(170, 23)

# metric

In [7]:
y_pred = pd.DataFrame(y_pred,index=[*range(len(X_test))],columns=y_cols)
y_pred.to_excel('test_170_y_pred.xlsx')
y_pred.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.203653,0.819773,0.162815,3.601341,2.287190,3.558242,0.671906,8.027853,6.380872,9.598312,3.297430,8.046802,8.438970,7.059808,5.898356,5.231463,9.390811,4.992514,3.515831,1.149199,5.289074,0.680578,0.697207
1,0.139352,0.104192,0.102327,3.308866,1.101464,2.172200,0.400289,10.542946,7.665590,6.351773,2.890178,9.757593,10.330828,4.990572,5.599922,6.804547,10.797002,4.466143,3.894958,1.195522,6.077098,0.601914,0.704724
2,0.127107,0.090924,0.104199,3.324569,1.111369,2.099061,0.373146,10.585086,7.645603,6.088890,2.801484,9.843629,10.421714,4.789957,5.451794,6.921744,10.855951,4.338547,3.932949,1.333048,6.311257,0.602323,0.845649
3,0.117018,0.090114,0.103043,3.450327,1.181302,2.107158,0.412057,10.573455,7.545799,6.058211,2.837872,9.906973,10.448506,4.828103,5.559909,6.941361,10.916151,4.370552,3.929210,1.209316,6.229677,0.548304,0.635580
4,0.153042,0.112315,0.102394,3.262811,1.117300,2.159636,0.408219,10.380412,7.477605,6.314454,2.893342,9.699408,10.235621,5.017221,5.702840,6.833485,10.830172,4.461537,4.028533,1.236815,6.243015,0.594478,0.735346


In [8]:
y_test.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.142,0.616,0.217,3.745,2.577,4.028,0.557,7.669,5.9900,10.206,3.075,7.920,8.036,7.039,5.438,5.263,9.537,4.877,3.481,1.088,5.652,0.618,0.597
1,0.122,0.078,0.029,3.955,1.563,2.407,0.400,10.016,7.2633,6.573,2.809,9.544,9.815,5.135,5.482,6.793,10.850,4.308,3.963,1.069,6.025,0.585,0.484
2,0.098,0.064,0.020,3.920,1.447,2.296,0.396,10.236,7.3480,6.259,2.797,9.655,9.986,4.942,5.453,6.947,10.996,4.269,3.997,1.034,6.107,0.543,0.456
3,0.122,0.085,0.021,3.900,1.437,2.274,0.397,10.259,7.3560,6.234,2.820,9.670,10.146,4.928,5.476,6.953,11.026,4.264,3.982,1.013,6.054,0.545,0.449
4,0.072,0.049,0.015,3.873,1.369,2.206,0.489,10.213,7.3190,6.009,2.968,9.564,10.052,4.769,6.584,6.866,10.892,4.169,4.227,0.930,5.828,0.519,0.463


In [9]:
N_col = ['C5N','C6N','C6A','C7N','C7A','C8N','C8A','C9N','C9A','C10N','C10A']
temp = pd.DataFrame()
temp['real_N+A'] = X_test['N+A'].values
temp['pred_N+A'] = y_pred[N_col].sum(axis=1).values
temp

,real_N+A,pred_N+A
0,40.133,40.133
1,32.175,32.175
2,31.428,31.428
3,31.390,31.390
4,32.418,32.418
...,...,...
165,53.841,53.841
166,53.442,53.442
167,52.549,52.549
168,49.827,49.827


In [0]:
def mape(a, b):
    a = np.array(a)
    b = np.array(b)
    mask = a != 0
    return (np.fabs(a-b)/a)[mask].mean()

def metric(y_t,y_p,name):
  res = {'R2' : max(r2_score(y_t,y_p), 0),
         'RMSE': sqrt(mean_squared_error(y_t,y_p)),
         'MAPE': mape(y_t,y_p)}
  return pd.DataFrame(res,index=[name])

In [0]:
res = pd.DataFrame()
for y_name in y_cols:
  res = res.append(metric(y_test[y_name],y_pred[y_name],y_name))
res.loc['AVG'] = res.mean()

In [12]:
res.style.background_gradient()

,R2,RMSE,MAPE
C5NP,0.873970,0.303383,0.286515
C5IP,0.747563,0.257107,0.477441
C5N,0.791447,0.062518,2.210832
C6NP,0.268807,0.345953,0.063609
C6IP,0.881107,0.360416,0.093511
C6N,0.887214,0.556157,0.052335
C6A,0.491823,0.121793,0.173974
C7NP,0.971811,0.369247,0.043671
C7IP,0.852935,0.423758,0.049249
C7N,0.982486,0.551507,0.033406


# check 是否 pass

In [13]:
c1 = (len(res.loc[res.RMSE > 0.6]) == 0)
c2 = (res.loc['AVG','RMSE'] < 0.4)
print(c1,c2)

True True


# model save

In [14]:
joblib.dump(model,'model.pkl')
print('save done')

save done
